In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

In [5]:
df = pd.read_csv('data_modified.csv', low_memory=False)
df = df.drop(columns=['Unnamed: 0', 'Description', 'Reference number', 'Location', 'Functions'])
df = df.dropna()
print(df.shape)
df.head()

(46090, 29)


,Bracelet material,Brand,Case diameter,Case material,Condition,Dial,Gender,Model,Movement,Scope of delivery,...,Chronograph,GMT,Tachymeter,Moon phase,Annual calendar,Perpetual calendar,Tourbillon,Flyback,Alarm,Country
0,Steel,Bulgari,41.0,Steel,New,Black,Men's watch/Unisex,Octo,Automatic,"Original box, original papers",...,0,0,0,0,0,0,0,0,0,United States of America
1,Steel,Tissot,39.0,Steel,New,Mother of pearl,Men's watch/Unisex,PR 100,Quartz,"Original box, no original papers",...,0,0,0,0,0,0,0,0,0,Japan
2,Leather,Mido,41.0,Steel,New,Silver,Men's watch/Unisex,Belluna,Automatic,"Original box, original papers",...,0,0,0,0,0,0,0,0,0,Japan
3,Steel,Mido,42.0,Steel,New,Black,Men's watch/Unisex,Multifort GMT,Automatic,"Original box, original papers",...,0,0,0,0,0,0,0,0,0,Japan
4,Leather,Seiko,42.0,Steel,Unworn,Black,Men's watch/Unisex,Kinetic,Quartz,"Original box, original papers",...,0,0,0,0,0,0,0,0,0,Japan


# 1. Exploratory analysis

To see how suitable some of the features are to differentiate between different watch brands, I examine their distribution between the top 10 brands by number of offerings. <br>
I use the odds-ratio (OR) to determine how the values of a features impact the likelihood that a watch belongs to a certain brand. <br> 
The odds ratio is defined as the odds that a watch is of a given brand divided by the odds that a watch is of a given brand when the feature has a certain value. For example, if the brand is 'Omega' and the value of the feature 'Movement' is 'Manual winding', an Odds-ratio of 5 implies that a watch with a hand-wound movement is 5 times more likely to be an Omega compared to the base probability.

\begin{equation*}
OR^{>1} = \frac{R(Brand)}{R(Brand^{feat})}
\end{equation*}

A given feature-value can also decrease the odds of a certain brand to be present. The odds-ratio would then be between 0 and 1. <br>
To make this case more comparable, I modify Odds-ratios below 1 by taking the negative reciprocal. <br>
To follow the example above, if the brand is 'Rolex' and the value of the feature 'Movement' is 'Manual winding', an Odds-ratio of -4 implies that a watch with a hand-wound movement is 4 times less likely to be a Rolex compared to the base probability.

\begin{equation*}
OR^{<1} = -\frac{1}{OR^{>1}}
\end{equation*}

Lastly, there are also cases where a given brand never has a certain feature-value. Or alternatively, a given feature-value is only ever displayed by one specific brand. <br>
The odds-ratio would be zero in this case. I replace the odds-ratio with -999 or 999 respectively, to make this information more visible.

The individual features will probably increase their information value significantly when combined with other features. <br>

In [6]:
## get the ten most frequent brands
print(df['Brand'].value_counts()[0:10])
top_brands = df['Brand'].value_counts()[0:10]

Rolex             13050
Omega              4438
Cartier            2107
Breitling          1943
Seiko              1917
TAG Heuer          1715
Longines           1484
Hublot             1227
Patek Philippe     1167
Tudor              1138
Name: Brand, dtype: int64


In [7]:
## calculate odds-ratio based on the formula above
def get_brand_dist(feature):
    t = pd.DataFrame(columns=df[feature].unique(), index=top_brands.index.values)
    
    for brand in top_brands.index.values:
        odd1 = len(df[df['Brand'] == brand]) / len(df[(df['Brand'].isin(top_brands.index.values)) & (df['Brand'] != brand)])
        r = []
        for value in df[feature].unique():
            ## exception if none of the top-10 brands have a certain feature value
            try:
                target = len(df[(df['Brand'] == brand) & (df[feature] == value)])
                rest = len(df[(df['Brand'].isin(top_brands.index.values)) & (df['Brand'] != brand) & (df[feature] == value)])
                
                if target == 0:
                    x = -999
                elif rest == 0:
                    x = 999
                else:
                    x = (target/rest)/odd1
                    if x < 1:
                        x = -1/x
            except:
                x = 0
            r.append(x)
        t.loc[brand] = r
    
    t = t.apply(pd.to_numeric)
    return(t)

In [8]:
## apply styling to the dataframe, red hues for reduced odds, green hues for increased odds
def centered_background_gradient(s):
    cmap = plt.cm.get_cmap('RdYlGn')
    norm = plt.colors.TwoSlopeNorm(vmin=-30, vcenter=0, vmax=30)
    return ['background-color: {:s}'.format(plt.colors.to_hex(c.flatten())) for c in cmap(norm(s.values))]

## 1.1 Movement
The type of movement offers some differentiation between brands. <br>
Automatic movements are the default, so the odds ratios are close to 1 in most cases. <br>
A quartz movement significantly increases the odds that a watch is made by Cartier. In contrast to this, Rolex and its daughter company Tudor have almost no quartz watches in their portfolio (Rolex has only produced a single quartz model in their history). <br>
A manually-wound watch is more likely to be an Omega, famous for their hand-wound Speedmaster, or one of the many hand-wound dress watches by Patek Phillipe.

In [9]:
get_brand_dist('Movement').style.apply(centered_background_gradient)

,Automatic,Quartz,Manual winding
Rolex,1.382586,-33.796308,-4.582402
Omega,-1.217173,-1.026445,5.880307
Cartier,-2.020800,5.357727,1.356001
Breitling,1.015900,1.110050,-1.904558
Seiko,-1.091823,1.911581,-2.849922
TAG Heuer,-1.327220,3.351334,-8.147018
Longines,-1.262322,3.002324,-12.024807
Hublot,1.099246,-1.617205,-2.523384
Patek Philippe,-1.259966,-1.249786,5.205007
Tudor,1.210269,-14.538412,-15.974232


## 1.2 Crystal
As with the movement, the default value, a sapphire crystal, offers little information for separating brands. <br>
Mineral glass is rarely found in the second most expensive brand on this list, Rolex. The use of mineral glass increases the odds that a watch is made by Seiko, the most affordable brand on this list fourty-fold. <br>
Plexiglass is used by most manufacturers, mostly in vintage models. Hublot, the youngest brand on the list does not have a single watch with a plexiglass crystal, signified by the OR of -999.

In [13]:
get_brand_dist('Crystal').style.apply(centered_background_gradient)

,Sapphire crystal,Mineral Glass,Plexiglass
Rolex,1.042205,-53.133080,1.190534
Omega,-1.237836,-1.429393,4.054840
Cartier,1.088532,1.321508,-15.134645
Breitling,1.106615,-2.568377,-5.110546
Seiko,-1.465228,39.089873,-2.448902
TAG Heuer,1.118499,-9.176062,-5.608470
Longines,1.108028,-11.840151,-3.630439
Hublot,1.137017,-12.951172,-999.000000
Patek Philippe,1.101876,-2.272149,-5.005814
Tudor,1.038375,-8.971427,-1.070825


## 1.3 Materials
Case and bracelet materials are very useful for separating brands. <br>
While common materials like steel and yellow gold are used by all brands to some extent, there are a lot of materials that are used by only a few manufacturers. For most materials you have at least one brand with a very high odds-ratio and multiple ones with the minimum value of -999. <br>
For example only five brands use platinum cases and only two platinum bracelets. <br>
Aluminium cases are used exclusively by Seiko which makes classification easy in this case.

In [14]:
get_brand_dist('Case material').style.apply(centered_background_gradient)

,Steel,Gold/Steel,Yellow gold,Titanium,Platinum,Rose gold,White gold,Ceramic,Bronze,Carbon,Plastic,Silver,Aluminum,Other
Rolex,-1.088768,2.996996,1.688069,-999.000000,1.545805,-1.561101,1.445276,-176.680672,-999.000000,-999.000000,-999.000000,-11.677171,-999,-999
Omega,1.221180,-1.620956,-2.103486,-1.090666,-9.709777,-1.743572,-5.956096,1.275769,-1.964937,-2.843988,-9.479960,-4.050528,-999,-999
Cartier,-1.351104,1.278065,2.895371,-999.000000,-3.151608,1.499097,2.487399,-999.000000,-999.000000,-5.177642,-999.000000,25.085234,-999,-999
Breitling,1.135892,-1.212479,-4.125725,1.989903,-999.000000,-1.499515,-999.000000,-31.990051,-999.000000,3.007401,392.465775,-6.673193,-999,-999
Seiko,1.372111,-16.260540,-13.989780,2.444603,-999.000000,-30.334925,-65.032474,-4.446582,-999.000000,-999.000000,-999.000000,-999.000000,999,-999
TAG Heuer,1.359211,-6.708364,-31.157450,1.562218,-999.000000,-11.513821,-999.000000,1.101679,-1.807102,1.277013,-3.313020,-999.000000,-999,-999
Longines,1.265969,-1.657053,-3.010633,-3.705433,-999.000000,-1.107321,-999.000000,-24.042227,2.072247,-999.000000,-999.000000,-999.000000,-999,-999
Hublot,-4.789243,-6.759063,-6.231453,18.228775,-7.245312,6.317452,-20.295349,40.715981,-2.584585,51.494110,-999.000000,-999.000000,-999,-999
Patek Philippe,-2.699185,-15.022548,3.787311,-999.000000,17.480287,5.237372,10.033780,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999,-999
Tudor,1.167833,-2.550316,-8.984485,1.785218,-999.000000,-999.000000,-999.000000,2.863240,87.515943,-999.000000,-999.000000,7.827815,-999,-999


In [15]:
get_brand_dist('Bracelet material').style.apply(centered_background_gradient)

,Steel,Leather,Gold/Steel,Rubber,Titanium,Yellow gold,Platinum,Textile,Rose gold,White gold,Ceramic,Aluminium,Silver
Rolex,1.315856,-7.520352,4.640224,-6.424810,-999.000000,3.874265,36.329195,-98.240546,1.628248,3.885162,-999.000000,-999,-1.903887
Omega,1.143319,1.481145,-3.487622,-1.369212,1.489328,-3.493221,-999.000000,1.497216,-2.490347,-2.979416,-4.481435,-999,2.320685
Cartier,-1.698465,2.296879,1.332653,-2.183208,-999.000000,1.924279,-999.000000,-4.105666,1.025118,1.903790,-999.000000,-999,-999.000000
Breitling,-1.311968,1.720905,-4.456889,3.143727,2.076538,-12.471697,-999.000000,2.386470,-3.013256,-999.000000,-1.788691,-999,2.422628
Seiko,1.172581,-1.061145,-37.850844,1.187732,7.440884,-999.000000,-999.000000,6.015712,-999.000000,-999.000000,-999.000000,-999,5.898592
TAG Heuer,1.155957,1.286706,-7.157415,1.442171,1.020784,-12.750108,-999.000000,1.185798,-999.000000,-999.000000,5.263784,-999,-999.000000
Longines,-1.066157,1.993425,-1.325010,-1.968369,-5.601236,-5.945927,-999.000000,-3.309038,1.094772,-999.000000,-1.344297,-999,-999.000000
Hublot,-98.238445,1.602713,-47.341690,11.023347,2.375987,-999.000000,-999.000000,3.550663,2.898426,-999.000000,43.476387,-999,-999.000000
Patek Philippe,-3.257621,3.448844,-11.203238,1.566756,-999.000000,1.619009,-1.112616,-5.187739,3.552332,1.955778,-999.000000,-999,-999.000000
Tudor,1.205073,1.090495,-2.538025,-2.869028,5.483252,-999.000000,-999.000000,3.413291,-999.000000,-999.000000,-999.000000,-999,-999.000000


## 1.4 Functions

In [13]:
get_brand_dist('Perpetual calendar').style.apply(centered_background_gradient)

,0,1
Rolex,-1.008350,2.074904
Omega,1.010889,-6.300822
Cartier,1.011667,-25.287902
Breitling,1.005986,-2.045112
Seiko,1.007589,-2.797278
TAG Heuer,1.012014,-999.000000
Longines,-1.011875,2.092630
Hublot,1.009230,-4.731344
Patek Philippe,-1.050010,6.033740
Tudor,1.011773,-999.000000


In [14]:
get_brand_dist('Tourbillon').style.apply(centered_background_gradient)

,0,1
Rolex,1.002340,-999.000000
Omega,1.001556,-999.000000
Cartier,1.001427,-999.000000
Breitling,1.001418,-999.000000
Seiko,1.001417,-999.000000
TAG Heuer,-1.014258,27.668610
Longines,1.001396,-999.000000
Hublot,-1.008036,8.952281
Patek Philippe,-1.000405,1.308754
Tudor,-1.000450,1.343446


# 2. Feature Engineering

In [15]:
df = df[df["Year of production"] > 1900]
df['Decade'] = 0
for i in range(190, 220):
    start = i * 10
    end = start + 10
    df.loc[(df['Year of production'] >= start) & (df['Year of production'] < end), 'Decade'] = '{}s'.format(start)

<ipython-input-15-4fa1da424169>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Decade'] = 0
C:\Users\simon\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\simon\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [16]:
df = df.loc[df['Case diameter'] > 14]
df['Diameter_group'] = 0
for i in range(0, 24):
    start = 14 + i*2
    end = start + 1
    df.loc[(df['Case diameter'] >= start) & (df['Case diameter'] <= end), 'Diameter_group'] = '{}-{}'.format(start, end)

In [12]:
keep = df['Brand'].value_counts()
keep = keep[keep>4]
keep = keep.index.get_level_values(0).astype(str).values
df = df[df['Brand'].isin(keep)]

In [13]:
df = df.dropna()

# 3. Feature Based Classification

In [14]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, GridSearchCV, ParameterGrid
from category_encoders.binary import BinaryEncoder
from category_encoders.target_encoder import TargetEncoder
from catboost import CatBoostClassifier
import time

In [15]:
categorical_var = ['Bracelet material', 'Case material',
       'Condition', 'Dial', 'Gender', 'Movement', 'Scope of delivery',
       'private_seller', 'Crystal', 'Decade', 'Diameter_group', 'Country']

In [16]:
functions = ['Date', 'Weekday', 'Month', 'Year', 'Chronograph', 'GMT', 
             'Tachymeter', 'Moon phase', 'Annual calendar', 'Perpetual calendar',
             'Tourbillon', 'Flyback', 'Alarm']

In [17]:
X = df[categorical_var + functions]
y = df['Brand']

## 3.1 First test of different classifiers

In [14]:
%%time

transformers = [
    ColumnTransformer([('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_var)], remainder='passthrough'),
    ColumnTransformer([('binary', BinaryEncoder(), categorical_var)], remainder='passthrough'),
]

models = [
    LogisticRegression(random_state=13, max_iter=1000),
    KNeighborsClassifier(),
    RandomForestClassifier(random_state=13),
]


for transformer in transformers:
    print(transformer.transformers[0][0])
    for model in models:
        start_time = time.time()
        pipe = Pipeline([('tf', transformer), ('model', model)])
        scores = cross_val_score(pipe, X, y, cv=5, n_jobs=5)
        print('{}. Score: {}. Time: {}s'.format(model, np.mean(scores), time.time() - start_time))

binary
RandomForestClassifier(random_state=13). Score: 0.5998952206142036. Time: 10.925626516342163s
Wall time: 10.9 s


## 3.2 Hyperparameter tuning for Random Forest

In [17]:
%%time

params = [
    {'model__max_depth': [50, 100, 200, None],
     'model__max_features': ['log2', 'auto'],
     'model__min_samples_leaf': [1, 2],
     'model__min_samples_split': [2, 5, 10],
     'model__n_estimators': [100, 200, 500]}]


transformer = ColumnTransformer([
        ('cat', BinaryEncoder(), categorical_var),
    ], remainder='passthrough')


pipe = Pipeline([('tf', transformer), ('model',RandomForestClassifier(random_state=13, n_jobs=5))])

grid = RandomizedSearchCV(estimator=pipe,
             param_distributions=params, n_jobs=1, n_iter=100, cv=3, verbose=1, random_state=13)

grid.fit(X, y)

print(grid.best_params_)
print(grid.best_score_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders

C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders

C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders

C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders

...

C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders

C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders

C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders

C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders

{'model__n_estimators': 200, 'model__min_samples_split': 5, 'model__min_samples_leaf': 1, 'model__max_features': 'auto', 'model__max_depth': None}
0.5914113107143263
Wall time: 32min 17s


In [18]:
transformer = ColumnTransformer([('binary', BinaryEncoder(), categorical_var)], remainder='passthrough')

pipe = Pipeline([('tf', transformer), ('model',RandomForestClassifier(random_state=13, n_jobs=5, 
                                       n_estimators= 200, 
                                       min_samples_split=5,
                                       min_samples_leaf=1,
                                       max_features='auto',
                                       max_depth = None))])


np.mean(cross_val_score(pipe, X, y, cv=5, n_jobs=5))

0.6024405788128746

## 3.3 Gradient boosting

In [15]:
%%time

params = [
    {'model__depth': [4, 6, 8]}]


transformer = ColumnTransformer([
        ('cat', BinaryEncoder(), categorical_var),
    ], remainder='passthrough')


pipe = Pipeline([('tf', transformer), ('model',CatBoostClassifier(random_state=13, task_type='GPU',loss_function='MultiClass', eval_metric = 'Accuracy', devices='0:1', early_stopping_rounds=10))])

grid = GridSearchCV(estimator=pipe,
             param_grid=params, n_jobs=1, cv=3, verbose=1)

grid.fit(X, y)

print(grid.best_params_)
print(grid.best_score_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Learning rate set to 0.130467
0:	learn: 0.3350302	total: 35.8ms	remaining: 35.8s
1:	learn: 0.3421439	total: 70.1ms	remaining: 35s
2:	learn: 0.3555556	total: 103ms	remaining: 34.2s
3:	learn: 0.3567303	total: 136ms	remaining: 33.9s
4:	learn: 0.3700441	total: 169ms	remaining: 33.6s
5:	learn: 0.3749714	total: 203ms	remaining: 33.7s
6:	learn: 0.3812041	total: 236ms	remaining: 33.4s
7:	learn: 0.3849894	total: 272ms	remaining: 33.7s
8:	learn: 0.3872736	total: 308ms	remaining: 33.9s
9:	learn: 0.3933105	total: 340ms	remaining: 33.7s
10:	learn: 0.4010442	total: 380ms	remaining: 34.2s
11:	learn: 0.4031326	total: 413ms	remaining: 34s
12:	learn: 0.4063306	total: 446ms	remaining: 33.9s
13:	learn: 0.4109969	total: 481ms	remaining: 33.8s
14:	learn: 0.4142274	total: 516ms	remaining: 33.9s
15:	learn: 0.4209170	total: 548ms	remaining: 33.7s
16:	learn: 0.4243433	total: 581ms	remaining: 33.6s
17:	learn: 0.4264643	total: 614ms	remaining: 33.5s
18:	learn: 0.4306086	total: 648ms	remaining: 33.4s
19:	learn: 0.

161:	learn: 0.6063306	total: 5.73s	remaining: 29.6s
162:	learn: 0.6072442	total: 5.76s	remaining: 29.6s
163:	learn: 0.6073748	total: 5.8s	remaining: 29.6s
164:	learn: 0.6073095	total: 5.83s	remaining: 29.5s
165:	learn: 0.6072769	total: 5.86s	remaining: 29.5s
166:	learn: 0.6084843	total: 5.9s	remaining: 29.4s
167:	learn: 0.6091043	total: 5.93s	remaining: 29.4s
168:	learn: 0.6092022	total: 5.96s	remaining: 29.3s
169:	learn: 0.6094958	total: 6s	remaining: 29.3s
170:	learn: 0.6095611	total: 6.03s	remaining: 29.2s
171:	learn: 0.6093979	total: 6.06s	remaining: 29.2s
172:	learn: 0.6099201	total: 6.1s	remaining: 29.1s
173:	learn: 0.6099527	total: 6.13s	remaining: 29.1s
174:	learn: 0.6102464	total: 6.17s	remaining: 29.1s
175:	learn: 0.6102464	total: 6.21s	remaining: 29.1s
176:	learn: 0.6107685	total: 6.25s	remaining: 29.1s
177:	learn: 0.6119106	total: 6.29s	remaining: 29s
178:	learn: 0.6128895	total: 6.33s	remaining: 29s
179:	learn: 0.6127916	total: 6.37s	remaining: 29s
180:	learn: 0.6133790	to

321:	learn: 0.6573340	total: 11.5s	remaining: 24.2s
322:	learn: 0.6572361	total: 11.5s	remaining: 24.2s
323:	learn: 0.6576277	total: 11.6s	remaining: 24.1s
324:	learn: 0.6575624	total: 11.6s	remaining: 24.1s
325:	learn: 0.6576603	total: 11.6s	remaining: 24s
326:	learn: 0.6578561	total: 11.7s	remaining: 24s
327:	learn: 0.6579866	total: 11.7s	remaining: 24s
328:	learn: 0.6578887	total: 11.7s	remaining: 23.9s
329:	learn: 0.6584435	total: 11.8s	remaining: 23.9s
330:	learn: 0.6584108	total: 11.8s	remaining: 23.8s
331:	learn: 0.6589982	total: 11.8s	remaining: 23.8s
332:	learn: 0.6587698	total: 11.9s	remaining: 23.8s
333:	learn: 0.6592919	total: 11.9s	remaining: 23.7s
334:	learn: 0.6595529	total: 11.9s	remaining: 23.7s
335:	learn: 0.6597814	total: 12s	remaining: 23.6s
336:	learn: 0.6603035	total: 12s	remaining: 23.6s
337:	learn: 0.6604014	total: 12s	remaining: 23.6s
338:	learn: 0.6609561	total: 12.1s	remaining: 23.5s
339:	learn: 0.6610214	total: 12.1s	remaining: 23.5s
340:	learn: 0.6610866	to

...

324:	learn: 0.7392969	total: 20.4s	remaining: 42.3s
325:	learn: 0.7399930	total: 20.4s	remaining: 42.3s
326:	learn: 0.7405369	total: 20.5s	remaining: 42.2s
327:	learn: 0.7410808	total: 20.6s	remaining: 42.1s
328:	learn: 0.7414941	total: 20.6s	remaining: 42.1s
329:	learn: 0.7415811	total: 20.7s	remaining: 42s
330:	learn: 0.7420814	total: 20.7s	remaining: 41.9s
331:	learn: 0.7423425	total: 20.8s	remaining: 41.8s
332:	learn: 0.7425600	total: 20.9s	remaining: 41.8s
333:	learn: 0.7429081	total: 20.9s	remaining: 41.7s
334:	learn: 0.7429516	total: 21s	remaining: 41.6s
335:	learn: 0.7434737	total: 21s	remaining: 41.6s
336:	learn: 0.7439523	total: 21.1s	remaining: 41.5s
337:	learn: 0.7446702	total: 21.2s	remaining: 41.4s
338:	learn: 0.7449748	total: 21.2s	remaining: 41.4s
339:	learn: 0.7452141	total: 21.3s	remaining: 41.3s
340:	learn: 0.7457797	total: 21.3s	remaining: 41.2s
341:	learn: 0.7463453	total: 21.4s	remaining: 41.2s
342:	learn: 0.7463888	total: 21.5s	remaining: 41.1s
343:	learn: 0.7464

485:	learn: 0.7820658	total: 31.1s	remaining: 32.9s
486:	learn: 0.7821963	total: 31.2s	remaining: 32.9s
487:	learn: 0.7824791	total: 31.3s	remaining: 32.8s
488:	learn: 0.7824139	total: 31.3s	remaining: 32.8s
489:	learn: 0.7825444	total: 31.4s	remaining: 32.7s
490:	learn: 0.7828924	total: 31.5s	remaining: 32.6s
491:	learn: 0.7835233	total: 31.6s	remaining: 32.6s
492:	learn: 0.7837191	total: 31.6s	remaining: 32.5s
493:	learn: 0.7837191	total: 31.7s	remaining: 32.5s
494:	learn: 0.7836321	total: 31.8s	remaining: 32.4s
495:	learn: 0.7843717	total: 31.9s	remaining: 32.4s
496:	learn: 0.7844152	total: 31.9s	remaining: 32.3s
497:	learn: 0.7845458	total: 32s	remaining: 32.3s
498:	learn: 0.7846328	total: 32.1s	remaining: 32.2s
499:	learn: 0.7849591	total: 32.2s	remaining: 32.2s
500:	learn: 0.7846328	total: 32.2s	remaining: 32.1s
501:	learn: 0.7849809	total: 32.3s	remaining: 32s
502:	learn: 0.7853289	total: 32.4s	remaining: 32s
503:	learn: 0.7858728	total: 32.4s	remaining: 31.9s
504:	learn: 0.7861

645:	learn: 0.8139575	total: 41.3s	remaining: 22.6s
646:	learn: 0.8139140	total: 41.3s	remaining: 22.5s
647:	learn: 0.8139358	total: 41.4s	remaining: 22.5s
648:	learn: 0.8140446	total: 41.4s	remaining: 22.4s
649:	learn: 0.8142186	total: 41.5s	remaining: 22.3s
650:	learn: 0.8143274	total: 41.6s	remaining: 22.3s
651:	learn: 0.8144361	total: 41.6s	remaining: 22.2s
652:	learn: 0.8147624	total: 41.7s	remaining: 22.2s
653:	learn: 0.8148277	total: 41.8s	remaining: 22.1s
654:	learn: 0.8149147	total: 41.9s	remaining: 22s
655:	learn: 0.8150017	total: 41.9s	remaining: 22s
656:	learn: 0.8151323	total: 42s	remaining: 21.9s
657:	learn: 0.8153498	total: 42.1s	remaining: 21.9s
658:	learn: 0.8155456	total: 42.1s	remaining: 21.8s
659:	learn: 0.8155891	total: 42.2s	remaining: 21.7s
660:	learn: 0.8160677	total: 42.3s	remaining: 21.7s
661:	learn: 0.8163288	total: 42.3s	remaining: 21.6s
662:	learn: 0.8167638	total: 42.4s	remaining: 21.6s
663:	learn: 0.8169814	total: 42.5s	remaining: 21.5s
664:	learn: 0.8169

806:	learn: 0.8369300	total: 51.3s	remaining: 12.3s
807:	learn: 0.8370388	total: 51.4s	remaining: 12.2s
808:	learn: 0.8372346	total: 51.4s	remaining: 12.1s
809:	learn: 0.8372999	total: 51.5s	remaining: 12.1s
810:	learn: 0.8374304	total: 51.5s	remaining: 12s
811:	learn: 0.8378002	total: 51.6s	remaining: 11.9s
812:	learn: 0.8378437	total: 51.7s	remaining: 11.9s
813:	learn: 0.8378655	total: 51.7s	remaining: 11.8s
814:	learn: 0.8377349	total: 51.8s	remaining: 11.8s
815:	learn: 0.8380830	total: 51.8s	remaining: 11.7s
816:	learn: 0.8383223	total: 51.9s	remaining: 11.6s
817:	learn: 0.8383006	total: 52s	remaining: 11.6s
818:	learn: 0.8384093	total: 52s	remaining: 11.5s
819:	learn: 0.8382570	total: 52.1s	remaining: 11.4s
820:	learn: 0.8383658	total: 52.2s	remaining: 11.4s
821:	learn: 0.8386921	total: 52.2s	remaining: 11.3s
822:	learn: 0.8388444	total: 52.3s	remaining: 11.2s
823:	learn: 0.8390837	total: 52.3s	remaining: 11.2s
824:	learn: 0.8393230	total: 52.4s	remaining: 11.1s
825:	learn: 0.8395

966:	learn: 0.8552689	total: 1m 1s	remaining: 2.09s
967:	learn: 0.8554647	total: 1m 1s	remaining: 2.02s
968:	learn: 0.8555299	total: 1m 1s	remaining: 1.96s
969:	learn: 0.8556387	total: 1m 1s	remaining: 1.9s
970:	learn: 0.8558562	total: 1m 1s	remaining: 1.83s
971:	learn: 0.8559650	total: 1m 1s	remaining: 1.77s
972:	learn: 0.8560085	total: 1m 1s	remaining: 1.71s
973:	learn: 0.8563566	total: 1m 1s	remaining: 1.64s
974:	learn: 0.8564871	total: 1m 1s	remaining: 1.58s
975:	learn: 0.8564219	total: 1m 1s	remaining: 1.52s
976:	learn: 0.8567264	total: 1m 1s	remaining: 1.45s
977:	learn: 0.8566394	total: 1m 1s	remaining: 1.39s
978:	learn: 0.8567699	total: 1m 1s	remaining: 1.33s
979:	learn: 0.8568352	total: 1m 1s	remaining: 1.26s
980:	learn: 0.8569875	total: 1m 2s	remaining: 1.2s
981:	learn: 0.8569440	total: 1m 2s	remaining: 1.14s
982:	learn: 0.8569222	total: 1m 2s	remaining: 1.07s
983:	learn: 0.8571615	total: 1m 2s	remaining: 1.01s
984:	learn: 0.8570962	total: 1m 2s	remaining: 948ms
985:	learn: 0.

In [ ]:
%%time

transformer = ColumnTransformer([('binary', BinaryEncoder(), categorical_var)], remainder='passthrough')

pipe = Pipeline([('tf', transformer), ('model',CatBoostClassifier(random_state=13, depth = 6,
    loss_function='MultiClass', eval_metric = 'Accuracy', task_type='GPU', devices='0:1', early_stopping_rounds=10))])


np.mean(cross_val_score(pipe, X, y, cv=5, n_jobs=1))

# 4. Image based classification

In [34]:
df_words = pd.read_csv('word_clusters.csv', low_memory=False)
df = df.merge(df_words, on='id')

X = df[map(str, range(0, 389))]
y = df['Brand']

## 4.1 First test of different classifiers

In [24]:
%%time

models = [
    LogisticRegression(random_state=13, max_iter=1000),
    KNeighborsClassifier(),
    RandomForestClassifier(random_state=13),
]


for model in models:
    start_time = time.time()
    scores = cross_val_score(model, X, y, cv=5, n_jobs=5)
    print('{}. Score: {}. Time: {}s'.format(model, np.mean(scores), time.time() - start_time))

C:\Users\simon\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


LogisticRegression(max_iter=1000, random_state=13). Score: 0.5656471125313856. Time: 92.00567746162415s


C:\Users\simon\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


KNeighborsClassifier(). Score: 0.4074640493038119. Time: 12.727590084075928s


C:\Users\simon\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


RandomForestClassifier(random_state=13). Score: 0.55243095183748. Time: 35.154783487319946s
Wall time: 2min 19s


## 4.2 Hyperparameter tuning for random forest

In [16]:
%%time

params = [
    {'max_depth': [200, 500, None],
     'max_features': ['auto', 'log2'],
     'min_samples_leaf': [1, 2],
     'min_samples_split': [2, 20, 50],
     'n_estimators': [100, 200, 500]}]



grid = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=13, n_jobs=5),
             param_distributions=params, n_jobs=1, n_iter=100, cv=3, verbose=1, random_state=13)

grid.fit(X, y)

print(grid.best_params_)
print(grid.best_score_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
{'n_estimators': 200, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 200}
0.5628397402161429
Wall time: 52min 6s


In [19]:
%%time

model = RandomForestClassifier(random_state=13, n_jobs=5, 
                                       n_estimators= 500, 
                                       min_samples_split=20,
                                       min_samples_leaf=1,
                                       max_features='log2',
                                       max_depth = 200)


np.mean(cross_val_score(model, X, y, cv=5, n_jobs=5))

C:\Users\simon\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Wall time: 1min 1s


0.5657384158867839

## 4.3 Gradient boosting

In [16]:
%%time


params = [
    {'depth': [4, 6, 8]}]


model = CatBoostClassifier(random_state=13, task_type='GPU',loss_function='MultiClass', eval_metric = 'Accuracy', devices='0:1', early_stopping_rounds=10)

grid = GridSearchCV(estimator=model,
             param_grid=params, n_jobs=1, cv=3, verbose=1)

grid.fit(X, y)

print(grid.best_params_)
print(grid.best_score_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Learning rate set to 0.129193
0:	learn: 0.3337670	total: 34.9ms	remaining: 34.9s
1:	learn: 0.3733137	total: 67.8ms	remaining: 33.8s
2:	learn: 0.3974526	total: 101ms	remaining: 33.6s
3:	learn: 0.4142984	total: 135ms	remaining: 33.7s
4:	learn: 0.4181333	total: 170ms	remaining: 33.9s
5:	learn: 0.4309388	total: 203ms	remaining: 33.7s
6:	learn: 0.4309388	total: 236ms	remaining: 33.5s
7:	learn: 0.4309388	total: 272ms	remaining: 33.7s
8:	learn: 0.4326508	total: 304ms	remaining: 33.4s
9:	learn: 0.4332329	total: 337ms	remaining: 33.4s
10:	learn: 0.4366911	total: 369ms	remaining: 33.2s
11:	learn: 0.4393960	total: 404ms	remaining: 33.2s
12:	learn: 0.4434705	total: 439ms	remaining: 33.3s
13:	learn: 0.4486407	total: 474ms	remaining: 33.4s
14:	learn: 0.4486407	total: 509ms	remaining: 33.4s
15:	learn: 0.4491885	total: 541ms	remaining: 33.3s
16:	learn: 0.4528179	total: 574ms	remaining: 33.2s
17:	learn: 0.4565158	total: 607ms	remaining: 33.1s
1

161:	learn: 0.5304047	total: 5.42s	remaining: 28s
162:	learn: 0.5304047	total: 5.46s	remaining: 28s
163:	learn: 0.5305074	total: 5.49s	remaining: 28s
164:	learn: 0.5306101	total: 5.53s	remaining: 28s
165:	learn: 0.5306786	total: 5.56s	remaining: 27.9s
166:	learn: 0.5308841	total: 5.59s	remaining: 27.9s
167:	learn: 0.5312265	total: 5.62s	remaining: 27.9s
168:	learn: 0.5314661	total: 5.66s	remaining: 27.8s
169:	learn: 0.5316716	total: 5.69s	remaining: 27.8s
170:	learn: 0.5319455	total: 5.72s	remaining: 27.7s
171:	learn: 0.5319113	total: 5.76s	remaining: 27.7s
172:	learn: 0.5322879	total: 5.8s	remaining: 27.7s
173:	learn: 0.5326988	total: 5.84s	remaining: 27.7s
174:	learn: 0.5328015	total: 5.88s	remaining: 27.7s
175:	learn: 0.5329042	total: 5.92s	remaining: 27.7s
176:	learn: 0.5329384	total: 5.95s	remaining: 27.7s
177:	learn: 0.5329384	total: 5.99s	remaining: 27.7s
178:	learn: 0.5331781	total: 6.03s	remaining: 27.7s
179:	learn: 0.5332466	total: 6.07s	remaining: 27.6s
180:	learn: 0.5333151

321:	learn: 0.5458467	total: 11.1s	remaining: 23.3s
322:	learn: 0.5459495	total: 11.1s	remaining: 23.3s
323:	learn: 0.5459495	total: 11.1s	remaining: 23.2s
324:	learn: 0.5461207	total: 11.2s	remaining: 23.2s
325:	learn: 0.5461891	total: 11.2s	remaining: 23.2s
326:	learn: 0.5462576	total: 11.2s	remaining: 23.1s
327:	learn: 0.5463946	total: 11.3s	remaining: 23.1s
328:	learn: 0.5463946	total: 11.3s	remaining: 23.1s
329:	learn: 0.5465658	total: 11.4s	remaining: 23.1s
330:	learn: 0.5466000	total: 11.4s	remaining: 23s
331:	learn: 0.5467712	total: 11.4s	remaining: 23s
332:	learn: 0.5469766	total: 11.5s	remaining: 22.9s
333:	learn: 0.5470451	total: 11.5s	remaining: 22.9s
334:	learn: 0.5471821	total: 11.5s	remaining: 22.9s
335:	learn: 0.5471478	total: 11.6s	remaining: 22.9s
336:	learn: 0.5471478	total: 11.6s	remaining: 22.8s
337:	learn: 0.5471821	total: 11.6s	remaining: 22.8s
338:	learn: 0.5472163	total: 11.7s	remaining: 22.8s
339:	learn: 0.5472848	total: 11.7s	remaining: 22.7s
340:	learn: 0.54

480:	learn: 0.5553653	total: 16.8s	remaining: 18.1s
481:	learn: 0.5553653	total: 16.8s	remaining: 18.1s
482:	learn: 0.5554681	total: 16.9s	remaining: 18.1s
483:	learn: 0.5556050	total: 16.9s	remaining: 18s
484:	learn: 0.5557420	total: 16.9s	remaining: 18s
485:	learn: 0.5559132	total: 17s	remaining: 18s
486:	learn: 0.5564268	total: 17s	remaining: 17.9s
487:	learn: 0.5564268	total: 17.1s	remaining: 17.9s
488:	learn: 0.5563925	total: 17.1s	remaining: 17.9s
489:	learn: 0.5563925	total: 17.1s	remaining: 17.8s
490:	learn: 0.5564268	total: 17.2s	remaining: 17.8s
491:	learn: 0.5564952	total: 17.2s	remaining: 17.8s
492:	learn: 0.5565637	total: 17.3s	remaining: 17.7s
493:	learn: 0.5566322	total: 17.3s	remaining: 17.7s
494:	learn: 0.5566322	total: 17.3s	remaining: 17.7s
495:	learn: 0.5570088	total: 17.4s	remaining: 17.6s
496:	learn: 0.5570088	total: 17.4s	remaining: 17.6s
497:	learn: 0.5570088	total: 17.4s	remaining: 17.6s
498:	learn: 0.5570088	total: 17.5s	remaining: 17.6s
499:	learn: 0.5571116	

643:	learn: 0.5653633	total: 22.6s	remaining: 12.5s
644:	learn: 0.5654318	total: 22.6s	remaining: 12.4s
645:	learn: 0.5653975	total: 22.7s	remaining: 12.4s
646:	learn: 0.5655002	total: 22.7s	remaining: 12.4s
647:	learn: 0.5656714	total: 22.7s	remaining: 12.4s
648:	learn: 0.5658084	total: 22.8s	remaining: 12.3s
649:	learn: 0.5658084	total: 22.8s	remaining: 12.3s
650:	learn: 0.5660138	total: 22.9s	remaining: 12.3s
651:	learn: 0.5660138	total: 22.9s	remaining: 12.2s
652:	learn: 0.5659111	total: 23s	remaining: 12.2s
653:	learn: 0.5659111	total: 23s	remaining: 12.2s
654:	learn: 0.5659111	total: 23.1s	remaining: 12.2s
655:	learn: 0.5659111	total: 23.1s	remaining: 12.1s
656:	learn: 0.5659454	total: 23.2s	remaining: 12.1s
657:	learn: 0.5659454	total: 23.2s	remaining: 12.1s
658:	learn: 0.5661508	total: 23.3s	remaining: 12s
659:	learn: 0.5661850	total: 23.3s	remaining: 12s
660:	learn: 0.5662877	total: 23.4s	remaining: 12s
661:	learn: 0.5662535	total: 23.4s	remaining: 12s
662:	learn: 0.5663562	to

...

278:	learn: 0.5650080	total: 32.8s	remaining: 1m 24s
279:	learn: 0.5650765	total: 33s	remaining: 1m 24s
280:	learn: 0.5650536	total: 33.1s	remaining: 1m 24s
281:	learn: 0.5652134	total: 33.2s	remaining: 1m 24s
282:	learn: 0.5652362	total: 33.3s	remaining: 1m 24s
283:	learn: 0.5653047	total: 33.4s	remaining: 1m 24s
284:	learn: 0.5654189	total: 33.6s	remaining: 1m 24s
285:	learn: 0.5655102	total: 33.7s	remaining: 1m 24s
286:	learn: 0.5659210	total: 33.8s	remaining: 1m 23s
287:	learn: 0.5658297	total: 33.9s	remaining: 1m 23s
288:	learn: 0.5659667	total: 34.1s	remaining: 1m 23s
289:	learn: 0.5662634	total: 34.2s	remaining: 1m 23s
290:	learn: 0.5662406	total: 34.3s	remaining: 1m 23s
291:	learn: 0.5664917	total: 34.5s	remaining: 1m 23s
292:	learn: 0.5667428	total: 34.6s	remaining: 1m 23s
293:	learn: 0.5668112	total: 34.7s	remaining: 1m 23s
294:	learn: 0.5667884	total: 34.8s	remaining: 1m 23s
295:	learn: 0.5668569	total: 34.9s	remaining: 1m 23s
296:	learn: 0.5669482	total: 35s	remaining: 1m 2

434:	learn: 0.5781328	total: 52.8s	remaining: 1m 8s
435:	learn: 0.5782241	total: 53s	remaining: 1m 8s
436:	learn: 0.5782926	total: 53.1s	remaining: 1m 8s
437:	learn: 0.5784296	total: 53.2s	remaining: 1m 8s
438:	learn: 0.5783839	total: 53.4s	remaining: 1m 8s
439:	learn: 0.5783839	total: 53.5s	remaining: 1m 8s
440:	learn: 0.5784524	total: 53.6s	remaining: 1m 7s
441:	learn: 0.5785894	total: 53.7s	remaining: 1m 7s
442:	learn: 0.5786122	total: 53.8s	remaining: 1m 7s
443:	learn: 0.5786578	total: 53.9s	remaining: 1m 7s
444:	learn: 0.5786807	total: 54.1s	remaining: 1m 7s
445:	learn: 0.5786578	total: 54.2s	remaining: 1m 7s
446:	learn: 0.5787491	total: 54.4s	remaining: 1m 7s
447:	learn: 0.5788176	total: 54.5s	remaining: 1m 7s
448:	learn: 0.5789546	total: 54.6s	remaining: 1m 6s
449:	learn: 0.5791372	total: 54.7s	remaining: 1m 6s
450:	learn: 0.5792513	total: 54.8s	remaining: 1m 6s
451:	learn: 0.5794111	total: 54.9s	remaining: 1m 6s
452:	learn: 0.5796622	total: 55s	remaining: 1m 6s
453:	learn: 0.57

594:	learn: 0.5871262	total: 1m 12s	remaining: 49.4s
595:	learn: 0.5871262	total: 1m 12s	remaining: 49.2s
596:	learn: 0.5872632	total: 1m 12s	remaining: 49.1s
597:	learn: 0.5873088	total: 1m 12s	remaining: 49s
598:	learn: 0.5873545	total: 1m 13s	remaining: 48.9s
599:	learn: 0.5874458	total: 1m 13s	remaining: 48.8s
600:	learn: 0.5875143	total: 1m 13s	remaining: 48.6s
601:	learn: 0.5875599	total: 1m 13s	remaining: 48.5s
602:	learn: 0.5875143	total: 1m 13s	remaining: 48.4s
603:	learn: 0.5876284	total: 1m 13s	remaining: 48.3s
604:	learn: 0.5876512	total: 1m 13s	remaining: 48.1s
605:	learn: 0.5877882	total: 1m 13s	remaining: 48s
606:	learn: 0.5878795	total: 1m 13s	remaining: 47.9s
607:	learn: 0.5879023	total: 1m 14s	remaining: 47.8s
608:	learn: 0.5879251	total: 1m 14s	remaining: 47.6s
609:	learn: 0.5880164	total: 1m 14s	remaining: 47.5s
610:	learn: 0.5880393	total: 1m 14s	remaining: 47.4s
611:	learn: 0.5880849	total: 1m 14s	remaining: 47.3s
612:	learn: 0.5880621	total: 1m 14s	remaining: 47.

751:	learn: 0.5940425	total: 1m 31s	remaining: 30.2s
752:	learn: 0.5940425	total: 1m 31s	remaining: 30.1s
753:	learn: 0.5939968	total: 1m 31s	remaining: 30s
754:	learn: 0.5940881	total: 1m 32s	remaining: 29.9s
755:	learn: 0.5941794	total: 1m 32s	remaining: 29.7s
756:	learn: 0.5942251	total: 1m 32s	remaining: 29.6s
757:	learn: 0.5940425	total: 1m 32s	remaining: 29.5s
758:	learn: 0.5939968	total: 1m 32s	remaining: 29.4s
759:	learn: 0.5939740	total: 1m 32s	remaining: 29.2s
760:	learn: 0.5940653	total: 1m 32s	remaining: 29.1s
761:	learn: 0.5940881	total: 1m 32s	remaining: 29s
762:	learn: 0.5941566	total: 1m 32s	remaining: 28.9s
763:	learn: 0.5941794	total: 1m 33s	remaining: 28.8s
764:	learn: 0.5942707	total: 1m 33s	remaining: 28.6s
765:	learn: 0.5943620	total: 1m 33s	remaining: 28.5s
766:	learn: 0.5944533	total: 1m 33s	remaining: 28.4s
767:	learn: 0.5944761	total: 1m 33s	remaining: 28.3s
768:	learn: 0.5944533	total: 1m 33s	remaining: 28.1s
769:	learn: 0.5945218	total: 1m 33s	remaining: 28s

907:	learn: 0.5996576	total: 1m 50s	remaining: 11.2s
908:	learn: 0.5998630	total: 1m 50s	remaining: 11s
909:	learn: 0.5998402	total: 1m 50s	remaining: 10.9s
910:	learn: 0.5998174	total: 1m 50s	remaining: 10.8s
911:	learn: 0.5998174	total: 1m 50s	remaining: 10.7s
912:	learn: 0.5998174	total: 1m 50s	remaining: 10.6s
913:	learn: 0.5998174	total: 1m 50s	remaining: 10.4s
914:	learn: 0.5998402	total: 1m 51s	remaining: 10.3s
915:	learn: 0.5999315	total: 1m 51s	remaining: 10.2s
916:	learn: 0.5999543	total: 1m 51s	remaining: 10.1s
917:	learn: 0.6000000	total: 1m 51s	remaining: 9.95s
918:	learn: 0.6000913	total: 1m 51s	remaining: 9.83s
919:	learn: 0.6000685	total: 1m 51s	remaining: 9.71s
920:	learn: 0.6001370	total: 1m 51s	remaining: 9.58s
921:	learn: 0.6001598	total: 1m 51s	remaining: 9.46s
922:	learn: 0.6002054	total: 1m 51s	remaining: 9.34s
923:	learn: 0.6002511	total: 1m 52s	remaining: 9.22s
924:	learn: 0.6003196	total: 1m 52s	remaining: 9.1s
925:	learn: 0.6003196	total: 1m 52s	remaining: 8.

In [18]:
%%time

model = CatBoostClassifier(random_state=13, depth = 8,
    loss_function='MultiClass', eval_metric = 'Accuracy', task_type='GPU', devices='0:1', early_stopping_rounds=10)


np.mean(cross_val_score(model, X, y, cv=5, n_jobs=1))

C:\Users\simon\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Learning rate set to 0.13409
0:	learn: 0.3603629	total: 113ms	remaining: 1m 52s
1:	learn: 0.4190824	total: 216ms	remaining: 1m 47s
2:	learn: 0.4390550	total: 323ms	remaining: 1m 47s
3:	learn: 0.4506677	total: 429ms	remaining: 1m 46s
4:	learn: 0.4549760	total: 540ms	remaining: 1m 47s
5:	learn: 0.4572016	total: 644ms	remaining: 1m 46s
6:	learn: 0.4661892	total: 747ms	remaining: 1m 45s
7:	learn: 0.4677300	total: 852ms	remaining: 1m 45s
8:	learn: 0.4719528	total: 955ms	remaining: 1m 45s
9:	learn: 0.4746348	total: 1.06s	remaining: 1m 45s
10:	learn: 0.4769459	total: 1.17s	remaining: 1m 45s
11:	learn: 0.4827950	total: 1.27s	remaining: 1m 44s
12:	learn: 0.4818820	total: 1.38s	remaining: 1m 44s
13:	learn: 0.4829948	total: 1.49s	remaining: 1m 44s
14:	learn: 0.4841075	total: 1.59s	remaining: 1m 44s
15:	learn: 0.4841931	total: 1.69s	remaining: 1m 44s
16:	learn: 0.4874743	total: 1.79s	remaining: 1m 43s
17:	learn: 0.4875029	total: 1.91s	remaining: 1m 44s
18:	learn: 0.4875314	total: 2.01s	remaining: 

158:	learn: 0.5514723	total: 18s	remaining: 1m 34s
159:	learn: 0.5519573	total: 18.1s	remaining: 1m 34s
160:	learn: 0.5521856	total: 18.2s	remaining: 1m 34s
161:	learn: 0.5522712	total: 18.3s	remaining: 1m 34s
162:	learn: 0.5522141	total: 18.4s	remaining: 1m 34s
163:	learn: 0.5525565	total: 18.5s	remaining: 1m 34s
164:	learn: 0.5529559	total: 18.6s	remaining: 1m 34s
165:	learn: 0.5529845	total: 18.7s	remaining: 1m 33s
166:	learn: 0.5532127	total: 18.8s	remaining: 1m 33s
167:	learn: 0.5535837	total: 18.9s	remaining: 1m 33s
168:	learn: 0.5535837	total: 19s	remaining: 1m 33s
169:	learn: 0.5535551	total: 19.1s	remaining: 1m 33s
170:	learn: 0.5535837	total: 19.2s	remaining: 1m 33s
171:	learn: 0.5537263	total: 19.3s	remaining: 1m 33s
172:	learn: 0.5539831	total: 19.5s	remaining: 1m 32s
173:	learn: 0.5540972	total: 19.6s	remaining: 1m 32s
174:	learn: 0.5541258	total: 19.7s	remaining: 1m 32s
175:	learn: 0.5542114	total: 19.8s	remaining: 1m 32s
176:	learn: 0.5543826	total: 19.9s	remaining: 1m 3

315:	learn: 0.5700468	total: 36.7s	remaining: 1m 19s
316:	learn: 0.5701324	total: 36.9s	remaining: 1m 19s
317:	learn: 0.5703036	total: 37s	remaining: 1m 19s
318:	learn: 0.5703892	total: 37.1s	remaining: 1m 19s
319:	learn: 0.5704177	total: 37.2s	remaining: 1m 19s
320:	learn: 0.5706174	total: 37.3s	remaining: 1m 18s
321:	learn: 0.5707030	total: 37.4s	remaining: 1m 18s
322:	learn: 0.5708457	total: 37.5s	remaining: 1m 18s
323:	learn: 0.5709028	total: 37.6s	remaining: 1m 18s
324:	learn: 0.5710454	total: 37.8s	remaining: 1m 18s
325:	learn: 0.5712166	total: 37.9s	remaining: 1m 18s
326:	learn: 0.5711881	total: 38s	remaining: 1m 18s
327:	learn: 0.5715019	total: 38.1s	remaining: 1m 18s
328:	learn: 0.5717587	total: 38.2s	remaining: 1m 17s
329:	learn: 0.5717873	total: 38.3s	remaining: 1m 17s
330:	learn: 0.5719585	total: 38.4s	remaining: 1m 17s
331:	learn: 0.5720441	total: 38.5s	remaining: 1m 17s
332:	learn: 0.5720155	total: 38.6s	remaining: 1m 17s
333:	learn: 0.5722152	total: 38.7s	remaining: 1m 1

473:	learn: 0.5835426	total: 55.4s	remaining: 1m 1s
474:	learn: 0.5834855	total: 55.5s	remaining: 1m 1s
475:	learn: 0.5834570	total: 55.6s	remaining: 1m 1s
476:	learn: 0.5835711	total: 55.8s	remaining: 1m 1s
477:	learn: 0.5837708	total: 55.9s	remaining: 1m 1s
478:	learn: 0.5840276	total: 56s	remaining: 1m
479:	learn: 0.5841703	total: 56.1s	remaining: 1m
480:	learn: 0.5840847	total: 56.3s	remaining: 1m
481:	learn: 0.5842844	total: 56.4s	remaining: 1m
482:	learn: 0.5846268	total: 56.5s	remaining: 1m
483:	learn: 0.5847980	total: 56.6s	remaining: 1m
484:	learn: 0.5848265	total: 56.7s	remaining: 1m
485:	learn: 0.5848551	total: 56.8s	remaining: 1m
486:	learn: 0.5849407	total: 57s	remaining: 60s
487:	learn: 0.5849977	total: 57.1s	remaining: 59.9s
488:	learn: 0.5851118	total: 57.2s	remaining: 59.8s
489:	learn: 0.5853401	total: 57.3s	remaining: 59.7s
490:	learn: 0.5853972	total: 57.4s	remaining: 59.6s
491:	learn: 0.5854542	total: 57.6s	remaining: 59.4s
492:	learn: 0.5855398	total: 57.7s	remaini

...

411:	learn: 0.5692194	total: 45.4s	remaining: 1m 4s
412:	learn: 0.5693906	total: 45.5s	remaining: 1m 4s
413:	learn: 0.5694191	total: 45.6s	remaining: 1m 4s
414:	learn: 0.5695617	total: 45.7s	remaining: 1m 4s
415:	learn: 0.5695332	total: 45.8s	remaining: 1m 4s
416:	learn: 0.5695047	total: 46s	remaining: 1m 4s
417:	learn: 0.5696473	total: 46.1s	remaining: 1m 4s
418:	learn: 0.5697044	total: 46.2s	remaining: 1m 4s
419:	learn: 0.5697615	total: 46.3s	remaining: 1m 3s
420:	learn: 0.5698185	total: 46.4s	remaining: 1m 3s
421:	learn: 0.5699612	total: 46.5s	remaining: 1m 3s
422:	learn: 0.5700753	total: 46.7s	remaining: 1m 3s
423:	learn: 0.5700753	total: 46.8s	remaining: 1m 3s
424:	learn: 0.5702180	total: 46.9s	remaining: 1m 3s
425:	learn: 0.5702180	total: 47s	remaining: 1m 3s
426:	learn: 0.5702465	total: 47.1s	remaining: 1m 3s
427:	learn: 0.5703036	total: 47.2s	remaining: 1m 3s
428:	learn: 0.5702751	total: 47.4s	remaining: 1m 3s
429:	learn: 0.5703036	total: 47.5s	remaining: 1m 2s
430:	learn: 0.57

572:	learn: 0.5796622	total: 1m 4s	remaining: 48.1s
573:	learn: 0.5798048	total: 1m 4s	remaining: 48s
574:	learn: 0.5798904	total: 1m 4s	remaining: 47.9s
575:	learn: 0.5798334	total: 1m 4s	remaining: 47.8s
576:	learn: 0.5799760	total: 1m 5s	remaining: 47.7s
577:	learn: 0.5800331	total: 1m 5s	remaining: 47.5s
578:	learn: 0.5800902	total: 1m 5s	remaining: 47.4s
579:	learn: 0.5800616	total: 1m 5s	remaining: 47.3s
580:	learn: 0.5800046	total: 1m 5s	remaining: 47.2s
581:	learn: 0.5800616	total: 1m 5s	remaining: 47.1s
582:	learn: 0.5801472	total: 1m 5s	remaining: 47s
583:	learn: 0.5802328	total: 1m 5s	remaining: 46.9s
584:	learn: 0.5802614	total: 1m 5s	remaining: 46.8s
585:	learn: 0.5803755	total: 1m 6s	remaining: 46.6s
586:	learn: 0.5804896	total: 1m 6s	remaining: 46.5s
587:	learn: 0.5805467	total: 1m 6s	remaining: 46.4s
588:	learn: 0.5805181	total: 1m 6s	remaining: 46.3s
589:	learn: 0.5805467	total: 1m 6s	remaining: 46.2s
590:	learn: 0.5806323	total: 1m 6s	remaining: 46.1s
591:	learn: 0.58

729:	learn: 0.5879936	total: 1m 22s	remaining: 30.4s
730:	learn: 0.5880221	total: 1m 22s	remaining: 30.3s
731:	learn: 0.5879936	total: 1m 22s	remaining: 30.2s
732:	learn: 0.5880792	total: 1m 22s	remaining: 30.1s
733:	learn: 0.5880792	total: 1m 22s	remaining: 30s
734:	learn: 0.5881077	total: 1m 22s	remaining: 29.9s
735:	learn: 0.5881648	total: 1m 22s	remaining: 29.8s
736:	learn: 0.5881648	total: 1m 23s	remaining: 29.6s
737:	learn: 0.5881933	total: 1m 23s	remaining: 29.5s
738:	learn: 0.5881933	total: 1m 23s	remaining: 29.4s
739:	learn: 0.5881648	total: 1m 23s	remaining: 29.3s
740:	learn: 0.5883075	total: 1m 23s	remaining: 29.2s
741:	learn: 0.5882789	total: 1m 23s	remaining: 29.1s
742:	learn: 0.5885072	total: 1m 23s	remaining: 29s
743:	learn: 0.5885072	total: 1m 23s	remaining: 28.9s
744:	learn: 0.5884501	total: 1m 23s	remaining: 28.7s
745:	learn: 0.5885357	total: 1m 24s	remaining: 28.6s
746:	learn: 0.5885072	total: 1m 24s	remaining: 28.5s
747:	learn: 0.5886213	total: 1m 24s	remaining: 28.

886:	learn: 0.5947272	total: 1m 40s	remaining: 12.7s
887:	learn: 0.5948128	total: 1m 40s	remaining: 12.6s
888:	learn: 0.5948984	total: 1m 40s	remaining: 12.5s
889:	learn: 0.5949555	total: 1m 40s	remaining: 12.4s
890:	learn: 0.5949840	total: 1m 40s	remaining: 12.3s
891:	learn: 0.5950696	total: 1m 40s	remaining: 12.2s
892:	learn: 0.5951552	total: 1m 40s	remaining: 12.1s
893:	learn: 0.5951552	total: 1m 40s	remaining: 12s
894:	learn: 0.5950982	total: 1m 40s	remaining: 11.8s
895:	learn: 0.5950696	total: 1m 41s	remaining: 11.7s
896:	learn: 0.5951552	total: 1m 41s	remaining: 11.6s
897:	learn: 0.5953835	total: 1m 41s	remaining: 11.5s
898:	learn: 0.5953549	total: 1m 41s	remaining: 11.4s
899:	learn: 0.5954120	total: 1m 41s	remaining: 11.3s
900:	learn: 0.5953835	total: 1m 41s	remaining: 11.2s
901:	learn: 0.5954120	total: 1m 41s	remaining: 11.1s
902:	learn: 0.5954405	total: 1m 41s	remaining: 10.9s
903:	learn: 0.5954405	total: 1m 41s	remaining: 10.8s
904:	learn: 0.5954405	total: 1m 42s	remaining: 1

0.5676557863501484

# 5 Ensemble models
## 5.1 Voting classifier

In [20]:
X = df
y = df['Brand']

In [22]:
%%time

n_cols = len(df[categorical_var + functions].columns)

m1 = RandomForestClassifier(random_state=13, n_jobs=5, n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features= 'auto', max_depth=None)
m2 = RandomForestClassifier(random_state=13, n_jobs=5, n_estimators=500, min_samples_split=20, min_samples_leaf=1, max_features= 'log2', max_depth=200)


pipe_1 = Pipeline([
    ('selector', ColumnTransformer([('selector', 'passthrough', categorical_var + functions)], remainder='drop')),
    ('encoder', ColumnTransformer([('encoder', BinaryEncoder(), list(range(0,n_cols)))], remainder='passthrough')),
    ('model', m1)
])

pipe_2 = Pipeline([
    ('selector', ColumnTransformer([('selector', 'passthrough', list(map(str, range(0, 389))))], remainder='drop')),
    ('model', m2)
])


clf = VotingClassifier(estimators=[('m1', pipe_1), ('m2', pipe_2)], voting='soft')

np.mean(cross_val_score(clf, X, y, cv=5, n_jobs=1))

C:\Users\simon\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_enco

Wall time: 2min 17s


0.70703035836567

## 5.2 Stacking Classifier

In [26]:
%%time

models = [('m1', pipe_1),
          ('m2', pipe_2)]

clf = StackingClassifier(estimators=models, final_estimator=LogisticRegression(n_jobs=5))

np.mean(cross_val_score(clf, X, y, cv=5, n_jobs=1))

C:\Users\simon\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_

C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\sklearn\model_sel

Wall time: 13min 9s


0.7341702807578179

# 6 Exkursions
## 6.1 Binary Classification

In [37]:
X = df
y = df['Brand'] == 'Rolex'

In [38]:
%%time

n_cols = len(df[categorical_var + functions].columns)

m1 = RandomForestClassifier(random_state=13, n_jobs=5, n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features= 'auto', max_depth=None)
m2 = RandomForestClassifier(random_state=13, n_jobs=5, n_estimators=500, min_samples_split=20, min_samples_leaf=1, max_features= 'log2', max_depth=200)


pipe_1 = Pipeline([
    ('selector', ColumnTransformer([('selector', 'passthrough', categorical_var + functions)], remainder='drop')),
    ('encoder', ColumnTransformer([('encoder', BinaryEncoder(), list(range(0,n_cols)))], remainder='passthrough')),
    ('model', m1)
])

pipe_2 = Pipeline([
    ('selector', ColumnTransformer([('selector', 'passthrough', list(map(str, range(0, 389))))], remainder='drop')),
    ('model', m2)
])

models = [('m1', pipe_1),
          ('m2', pipe_2)]

clf = StackingClassifier(estimators=models, final_estimator=LogisticRegression(n_jobs=5))

np.mean(cross_val_score(clf, X, y, cv=5, n_jobs=1))

C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\simon\anaconda3\lib\site-packages\category_encoders

Wall time: 6min 48s


0.9567222095412007

## 6.2 Model classification

In [ ]:
keep = df['Model'].value_counts()
keep = keep[keep>4]
keep = keep.index.get_level_values(0).astype(str).values
df = df[df['Model'].isin(keep)]